In [ ]:
import cv2 as cv
import numpy as np
import pyrealsense2 as rs
import time
import threading
import queue
from datetime import datetime

log_queue = queue.Queue()

def logger_worker():
    # Pembukaan 2 file untuk penyimpanan koordinat pada file txt
    with open("transform_coordinate_3D.txt", "a") as f3d, \
         open("transform_coordinate_stereo.txt", "a") as fst:
        while True:
            item = log_queue.get()           # Proses hanya akan berjalan ketika mendapat input dari queue
            if item is None:
                break                        # Proses penulisan berhenti ketika item mendapatkan sentinel None / looping berhenti
            ts, text_real_rs, text_transform_rs, text_real, text_trans = item
            # Penulisan koordinat pada setiap file
            f3d.write(f"{ts} — {text_real_rs} — {text_transform_rs} \n")
            fst.write(f"{ts} — {text_real} — {text_trans}\n")
            log_queue.task_done()   

def transform_coordinates(points3D, sensor_type): 
    # Pembuatan rotasi matriks yaitu x negatif 90 derajat serta z 90 derajat
    Rx_neg90 = np.array([
        [1,  0,  0],
        [0,  0,  1],
        [0,  -1,  0]
    ])
    Rz_90 = np.array([
        [0,  -1,  0],
        [1,  0,  0],     
        [0,  0,  1]
    ])
    # Perkalian rotas matriks dengan matriks z terletak pada posisi pertama dilanjutkan dengan matriks x
    R = np.dot(Rz_90, Rx_neg90)
    
    # Perhitungan matriks translasi berdasarkan jenis kamera yang digunakan (z,x,y)
    if sensor_type.lower() == 'stereo':
        # T = np.array([43, -6, 12]) LAMA
        T = np.array([41, -7, 20]) # BARU
    elif sensor_type.lower() == 'realsense':
        # T = np.array([50, 0, 13]) LAMA
        T = np.array([49, -3, 20]) # BARU
    else:
        raise ValueError("Sensor type must be 'stereo' or 'realsense'")
    
    # Konversi ke matriks 4x4 untuk menyesuaikan dengan ukuran matriks transformasi
    ones = np.ones((points3D.shape[0], 1))
    points_homogeneous = np.hstack((points3D, ones))
    
    # Pembuatan matriks transformasi
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = R  
    transformation_matrix[:3, 3] = T   
    
    # Transformasi matriks diaplikasikan pada titik koordinat x,y,z dari kamera
    transformed_points = (transformation_matrix @ points_homogeneous.T).T
    
    return transformed_points[:, :3]

def add_HSV_filter(frame):
    # Penggunaan Gaussian blur untuk mengurangi noise pada gambar
    blur = cv.GaussianBlur(frame, (5, 5), 0)
    hsv = cv.cvtColor(blur, cv.COLOR_BGR2HSV)
    # Range HSV untuk bola ping - pong
    lower_orange = np.array([9, 127, 142])
    upper_orange = np.array([22, 220, 255])
    # Pembuatan mask
    mask = cv.inRange(hsv, lower_orange, upper_orange)
    # Aplikasi erosi dan dilasi untuk memperbagus masking HSV
    mask = cv.erode(mask, None, iterations=2)
    mask = cv.dilate(mask, None, iterations=2)
    # Penggunaan morphologic untuk penghapusan noise tambahan pada gambar
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    return mask

def get_largest_bounding_box(mask):
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    largest_contour = max(contours, key=cv.contourArea)
    return cv.boundingRect(largest_contour)

def triangulate_centroid(cxL, cyL, cxR, cyR, cameraMatrixL, cameraMatrixR, rot, trans):
    # Pembuatan matriks proyeksi untuk kamera webcam kiri dan kanan
    projMatrixL = cameraMatrixL @ np.hstack((np.eye(3), np.zeros((3, 1))))
    projMatrixR = cameraMatrixR @ np.hstack((rot, trans.reshape(-1, 1)))
    # Penentuan titik sentroid 
    ptsL = np.array([[cxL], [cyL]], dtype=np.float32)
    ptsR = np.array([[cxR], [cyR]], dtype=np.float32)
    # Perhitungan triangulasi 
    pts4D = cv.triangulatePoints(projMatrixL, projMatrixR, ptsL, ptsR)
    pts3D = pts4D[:3] / pts4D[3]  # Konversi ke koordinat Euclidean
    # Konversi nilai x,y,z dengan dikalikan -1 untuk menyesuaikan dengan frame reference pada kamera intel realsense
    pts3D[2] *= -1
    pts3D[1] *= -1
    pts3D[0] *= -1
    pts3D[2] -= 7 #kalibrasi tambahan 
    return pts3D.flatten()

# Pengambilan nilai kamera matriks beserta nilai distorsinya dari program kalibrasi kamera
cameraMatrixL = np.load(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_L\old_camera_matrix_L.npy')
cameraMatrixR = np.load(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_R\old_camera_matrix_R.npy')
newCameraMatrixL = np.load(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_L\new_camera_matrix_L.npy')
newCameraMatrixR = np.load(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_R\new_camera_matrix_R.npy')
distL = np.load(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_L\distL.npy')
distR = np.load(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_R\distR.npy')
roi_L = np.load(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_L\roi_L.npy')
roi_R = np.load(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_R\roi_R.npy')

# Pembuatan matriks entrinsik untuk kamera webcam
rot = np.eye(3)
trans = np.array([[20, 0.6, 0.5]])

# Pembuatan threading untuk proses pencatatan koordinat
logger_thread = threading.Thread(
    target=logger_worker,
    daemon=True                      
)
logger_thread.start()

# Inisialisasi kamera webcam
capL = cv.VideoCapture(2)
capR = cv.VideoCapture(1)
capL.set(cv.CAP_PROP_FRAME_WIDTH, 640)
capL.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
capR.set(cv.CAP_PROP_FRAME_WIDTH, 640)
capR.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

# Inisialisasi kamera intel realsense
pipeline = rs.pipeline()
rs_config = rs.config()
rs_config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
rs_config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(rs_config)
profile = pipeline.get_active_profile()
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()  # in meters
color_intrinsics = profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()

num = 22  
recording = False

while True:
    # Proses kamera webcam kiri dan kanan
    retL, frameL = capL.read()
    retR, frameR = capR.read()
    if not retL or not retR:
        print("Error capturing stereo frames.")
        break

    # Proses penghapusan distorsi pada kamera webcam
    frame_undistortedL = cv.undistort(frameL, cameraMatrixL, distL, None, newCameraMatrixL)
    frame_undistortedR = cv.undistort(frameR, cameraMatrixR, distR, None, newCameraMatrixR)
    # Cropping hasil distorsi pada kamera webcam
    xL, yL, wL, hL = roi_L
    xR, yR, wR, hR = roi_R
    croppedL = frame_undistortedL[yL:yL+hL, xL:xL+wL]
    croppedR = frame_undistortedR[yR:yR+hR, xR:xR+wR]

    # Aplikasi filter HSV pada kamera webcam
    maskL = add_HSV_filter(croppedL)
    maskR = add_HSV_filter(croppedR)
    bboxL = get_largest_bounding_box(maskL)
    bboxR = get_largest_bounding_box(maskR)

    displayL = croppedL.copy()
    displayR = croppedR.copy()
    
    if bboxL is not None and bboxR is not None:
        # Penentuan sentroid dan bounding box untuk kamera webcam kiri dan kanan
        xL_box, yL_box, wL_box, hL_box = bboxL
        cxL = xL_box + wL_box // 2
        cyL = yL_box + hL_box // 2
        cv.rectangle(displayL, (xL_box, yL_box), (xL_box + wL_box, yL_box + hL_box), (0, 255, 0), 2)
        cv.circle(displayL, (cxL, cyL), 5, (0, 0, 255), -1)
        xR_box, yR_box, wR_box, hR_box = bboxR
        cxR = xR_box + wR_box // 2
        cyR = yR_box + hR_box // 2
        cv.rectangle(displayR, (xR_box, yR_box), (xR_box + wR_box, yR_box + hR_box), (0, 255, 0), 2)
        cv.circle(displayR, (cxR, cyR), 5, (0, 0, 255), -1)
        
        # Penentuan titik 3D dengan triangulasi melalui titik sentroid
        point3D = triangulate_centroid(cxL, cyL, cxR, cyR, cameraMatrixL, cameraMatrixR, rot, trans)
        # Transformasi untuk penyesuaian dengan frame dari lengan robot
        transformed_points = transform_coordinates((point3D.reshape(1, -1)),sensor_type='stereo')
        transformed_point=transformed_points[0]
        text_real = f"Real: X={point3D[0]:.2f}cm Y={point3D[1]:.2f}cm Z={point3D[2]:.2f}cm"
        text_trans = f"Robot: X={transformed_point[0]:.2f}cm Y={transformed_point[1]:.2f}cm Z={transformed_point[2]:.2f}cm"
        cv.putText(displayL, text_real, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv.LINE_AA)
        cv.putText(displayL, text_trans, (10, 60), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2, cv.LINE_AA)
    
    # Proses kamera intel real sense
    frames = pipeline.wait_for_frames()
    align = rs.align(rs.stream.color)
    aligned = align.process(frames)
    depth_frame = aligned.get_depth_frame()
    color_frame = aligned.get_color_frame()
    if not depth_frame or not color_frame:
        continue
    # Konversi frame dalam bentuk array untuk ekstraksi data piksel 
    rs_color = np.asanyarray(color_frame.get_data())
    # Aplikasi HSV filter pada kamera intel realsense
    rs_mask = add_HSV_filter(rs_color)
    rs_contours, _ = cv.findContours(rs_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    if rs_contours:
        largest_contour = max(rs_contours, key=cv.contourArea)
        x, y, w, h = cv.boundingRect(largest_contour)
        cx = x + w // 2
        cy = y + h // 2
        cv.rectangle(rs_color, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv.circle(rs_color, (cx, cy), 5, (0, 0, 255), -1)
        # Perhitungan nilai kedalaman dari piksel sentroid
        depth_value = depth_frame.get_distance(cx, cy)
        point_3D_rs = rs.rs2_deproject_pixel_to_point(color_intrinsics, [cx, cy], depth_value)
        point_3D_rs_cm = np.array(point_3D_rs) * 100 # Konversi ke satuan cm dari satuan m
        # Perhitungan transformasi untuk menyesuaikan dengan frame lengan robot
        transformed_rss = transform_coordinates((point_3D_rs_cm.reshape(1, -1)),sensor_type='realsense')
        transformed_rs=transformed_rss[0]
        text_real_rs = f"Real: X={point_3D_rs_cm[0]:.2f}cm Y={point_3D_rs_cm[1]:.2f}cm Z={point_3D_rs_cm[2]:.2f}cm"
        text_transform_rs = f"Robot: X={transformed_rs[0]:.2f}cm Y={transformed_rs[1]:.2f}cm Z={transformed_rs[2]:.2f}cm"
        cv.putText(rs_color, text_real_rs, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv.LINE_AA)
        cv.putText(rs_color, text_transform_rs, (10, 60), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2, cv.LINE_AA)
        
    # Dua kamera webcam dan satu kamera intel realsense ditampilkan pada layar laptop
    cv.imshow("Left Camera (Stereo)", displayL)
    cv.imshow("Right Camera (Stereo)", displayR)
    cv.imshow("Intel RealSense", rs_color)
    
    key = cv.waitKey(1) & 0xFF
    if key == ord('q'): # Untuk memberhentikan program
        break
    elif key == ord('s'): # Untuk mengambil foto ketika pengambilan data 
        cv.imwrite(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\hasil_koordinat_kamera\screenshot_' + str(num) + '.png', displayL)
        num += 1
        print("Screenshot saved.")
    elif key == ord('r'): # Memulai proses recording atau pencatatan koordinat dari kamera
        recording = True
        print("Recording start")
    elif key == ord('t'): # Memberhentikan proses recording atau pencatatan koordinat dari kamera
        recording = False
        print("Recording stop")
    if recording: 
        ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        # Proses queue dan threading dimulai 
        log_queue.put((ts, text_real_rs, text_transform_rs, text_real, text_trans))

log_queue.put(None) # Untuk memberhentikan queue
logger_thread.join() # Untuk memberhentikan threading

capL.release()
capR.release()
pipeline.stop()
cv.destroyAllWindows()
